# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [2]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [3]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

---

# Armado de ventanas

In [4]:
TAMAÑO_VENTANA = 2
FECHA_INICIO = 17

In [5]:
def etiquetar_ventanas(dataset,col_fecha):
    for ventana_nro in range(1,6):
        
        fecha_inicio = FECHA_INICIO + ventana_nro
        fecha_fin = fecha_inicio + TAMAÑO_VENTANA
        
        dataset["ventana_{}".format(ventana_nro)] = (((fecha_inicio) <= dataset[col_fecha].dt.day) & ((dataset[col_fecha].dt.day) <= (fecha_fin)))
    return dataset    

In [6]:
events = etiquetar_ventanas(events,"date")

---

# Creación de features

## Creación de sets de entrenamiento

In [7]:
def crear_targets(dataframe,col_dispositivo):
    entrenamientos = {}
    for ventana_nro in range(1,6):
        ent = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]][col_dispositivo].drop_duplicates().to_frame()
        ent.set_index(col_dispositivo, inplace=True)
        entrenamientos[ventana_nro] = ent
        
    return entrenamientos

In [8]:
Xs = crear_targets(events,"device_id")

---

## Función generadora de features

In [10]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    
    for ventana_nro in range(1, 6):
        
        ventana = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]]
        
        feature = generador_feature(ventana, ventana_nro)
        
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

In [15]:
#def calcular_features(ventanas,feature):
#    features = []
#    for x in range(5):
#        features.append(feature(ventanas[x]))
#    return features

In [16]:
#def mergear_features(targets,features):
#    for x in range(5):
#        targets[x] = targets[x].merge(features[x], how = 'left',left_index = True,right_index = True)


---

---

## Cantidad de eventos dentro de la ventna

In [10]:
def cantidad_eventos(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cant_eventos"})

In [11]:
generar_feature_en_ventanas(events,cantidad_eventos,Xs)

---

## Promedio de eventos por día

In [12]:
def prom_eventos(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id","dayofweek"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"mean"})
    feature["date"] = feature["date"].astype("int64")
    return feature.rename(columns={"date": "prom_eventos"})

In [13]:
generar_feature_en_ventanas(events, prom_eventos, Xs)

---

## Cantidad de eventos discrimando periodos

In [14]:
# Creo columnas para luego separar
events['hora'] = events['date'].dt.hour
events['dia'] = events['date'].dt.day
events.rename(columns = {"dayofweek":"dia_de_semana"},inplace = True)

In [15]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["eventos_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [16]:
generar_feature_en_ventanas(events, cantidad_apariciones_por_hora, Xs)

In [17]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dia_final = TAMAÑO_VENTANA + 2
    
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["eventos_dia_{}".format(dia) for dia in range(1,dia_final)]
    return apariciones_por_dia

In [18]:
generar_feature_en_ventanas(events, cantidad_apariciones_por_dia, Xs)

---

## Cantidad de veces que se realizó el evento más frecuente

In [21]:
def cantidad_maximo_evento(dataframe,nro_ventana):
    
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"max"})
    feature.rename(columns = {"date":"max_cant_eventos"},inplace = True)
    
    return feature
    

In [22]:
generar_feature_en_ventanas(events, cantidad_maximo_evento, Xs)

---

# ACA SE DEBE CONTINUAR

## Carga de datos trabajados

In [7]:
Xs = {}
for ventana_nro in range(1, 6):
    Xs[ventana_nro] = pd.read_pickle("../../features/events_train_ventana_{}.pkl".format(ventana_nro))


---

---

---

## Cuantos eventos comparten la primera posición de evento más frecuente

In [8]:
def cantidad_eventos_maximos(dataframe,nro_ventana):
    
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    idx = feature.groupby(['device_id'])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    feature = feature.groupby("device_id").agg({"date":"count"})
    feature.rename(columns = {"date":"cant_eventos_maximos"},inplace = True)
    
    return feature

In [11]:
generar_feature_en_ventanas(events, cantidad_eventos_maximos, Xs)

---

## Tiempo en segundos desde 't0' hasta hora más frecuente

In [13]:
events["hora"] = events["date"].dt.hour


In [1]:
def tiempo_hasta_hora_frecuente(dataframe,nro_ventana):
    segundos_en_hora = 60*60
    
    feature = dataframe.groupby(["device_id","hora"]).agg({"date":"count"})
    idx = feature.groupby(["device_id"])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    
    feature = feature.reset_index().groupby("device_id").agg({"hora":"min"})
    feature["hora"] = feature["hora"] * segundos_en_hora
    feature.rename(columns = {"hora":"segs_hasta_hora_freq"},inplace = True)
    
    return feature
    
    

In [18]:
generar_feature_en_ventanas(events,tiempo_hasta_hora_frecuente, Xs)

---

# Guardado de los sets de entrenamiento

In [21]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/events_train_ventana_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/events_train_ventana_{}.csv".format(ventana_nro))

---

### Propuesta: Hay valores repetidos, podría quedarme con la hora más cercana a las 00 hs y calcular el tiempo que falta desde t0 a esa hora. Además de indicar la cantidad de eventos registrados a esa hora, y la cantidad de horas que comparten el primer lugar.

### Propuesta: En vez de indicar valores categoricos como aplicación más usada, evento más registrado, se podría indicar cual es su valor de frecuencia, ejemplo:
#### __Freq_ev_popular__ : 53
#### __Freq_app_popular__ : 12

### Propuesta : En los valores en donde se comparten horas, eventos, aplicaciones, etc, como los más vistos, se podría indicar cual es la cantidad de valores que comparten ese puesto, ejemplo:

#### __cant_horas_mas_activas__: 2
#### __cant_events_en_horas_mas_activas__ : 10